# ABOUT

__Author__: Pat McCornack

__Date__: 4/9/24

__Purpose__: Train model to update FVT using most recent FDist data. 

------

In [1]:
import os
import datetime as dt
from joblib import dump
from itertools import combinations

import numpy as np
import pandas as pd
import geopandas as gpd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.preprocessing import TargetEncoder, StandardScaler
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier


import matplotlib.pyplot as plt




## Define Paths
Define set of filepaths to conveniently switch between working off local files or the PNNL drive. Set active_data_dir to either local_data_dir or pnnl_data_dir depending on which you're working off of. 

In [2]:
local_root_dir = r"C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\Fuel Attributes Model"
pnnl_root_dir = r"\\pnl\projects\BPAWildfire\data\Landfire\fuels_modeling\Fuel Attributes Model"

# Define which data directory to work off of
active_root_dir = local_root_dir

paths_dict = {
    "out_base_dir" : os.path.join(active_root_dir, r"model_outputs\tabular"),  # Where to save outputs 
    "ref_data_dir" : os.path.join(active_root_dir, r"..\LF_raster_data\_tables"),  # Location of LF csvs (e.g. LF22_FVT_230.csv)
    "sample_points_dir" : os.path.join(active_root_dir, r"data\sample_points"),  # Location of shapefile to train model on
    "sample_points_fname" : "sample_points_4-17-24_200k_Disturbed.shp",  # Name of shapefile to train model on
    "runs_dict_fpath" : os.path.join(active_root_dir, r"data\runs_dict.csv"),  # Used to test different sets of predictors 
    "model_dir" : os.path.join(active_root_dir, "models")
}




# __Functions__
----


## __Create a Directory to Output Modeling Results__
Names the output directory using the datetime that the script was run. 
Returns the name of the directory. The returned directory is used to output the trained model and/or results. 

In [3]:
def make_dir(base_dir, new_dir_name='model_results'):
        """
        Returns path to a directory created at the specified base_dir location. 

        The name of the created directory can optionally be specified using the dir_name argument. 
        """

        datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
        output_dir = os.path.join(base_dir, new_dir_name + "_" + datetime)

        os.makedirs(output_dir)
        return output_dir

## __Pre-Process the Data__

### __Create Data Dictionaries to Append Features__
Some features are separate attributes of the LANDFIRE dataset (e.g. BPS Fire Regime) and others are useful for results analysis (e.g. FDst attributes). These can be mapped to points using LANDFIRE CSVs. The below creates dictionaries to perform that mapping. 

This function is called by join_features.

In [8]:
def read_ref_data(ref_data_dir=paths_dict["ref_data_dir"]):
    """
    Returns a dictionary of dictionaries of mappings between LANDFIRE raster values and other attributes associated with those values. 
    """
    data_dir = ref_data_dir
    BPS_fname = "LF20_BPS_220.csv"
   
    # Create empty dictionary
    LF_ref_dicts = {}

    # Get BPS reference dictionary
    BPS_df = pd.read_csv(os.path.join(data_dir, BPS_fname))
    LF_ref_dicts["BPS_NAME"] = dict(BPS_df[['VALUE', 'BPS_NAME']].values)

    return LF_ref_dicts
                         
        

### __Append Features using Data Dictionaries__
Append in selected features using the LANDFIRE data dictionaries.

__Note:__ Items in feature_list must be in the source_layers dictionary. 

In [9]:
def join_features(sample_points, feature_list = ['BPS_NAME']):
    """
    Returns the sample_points layer with the features in feature_list appended. 

    Items in feature_list must be in the source_layers dictionary.  
    """
    
    LF_ref_dicts = read_ref_data()
    
    source_layers = {
        'BPS_NAME' : 'BPS', 
    }

    # Iterate through feature_list and append features to sample_points
    for feature in feature_list:
        sample_points[feature] = sample_points[source_layers[feature]].map(LF_ref_dicts[feature]).copy()

    return sample_points

### __Low Count Filter__
Some classes have exceedingly low representation in the dataset. In order to run train_test_split, the target class must have a count greater than 1. The following will filter out classes with counts of 1. Given the size of the dataset excluding these classes will not detrimentally impact model performance.

In [10]:
def low_count_filter(sample_points, target):  
    """
    Returns modified sample_points dataframe. Classes in the target feature that have counts of 1 are removed. 
    """  
    # Group by FVT
    LF22_FVT_counts = sample_points.groupby(target).count()

    # Identify FVTs with low observation counts
    low_count_FVT = LF22_FVT_counts[LF22_FVT_counts.iloc[:,0] < 5].index.tolist()

    # Remove those FVTs from sample_points
    sample_points = sample_points.loc[~sample_points[target].isin(low_count_FVT)]

    return sample_points

### __Prepare the Sample Points__
Before training the model, the sample points need to be filtered. Filtering steps are:
- Remove near points - defined as points that are less than 70m apart.
- Remove Null points (-9999/-1111 values) - these will not be updated in the final raster. 
- Remove undisturbed values - the model is intended to update points that have been disturbed in the last 10 years. 
- Remove points with agricultural or developed FVT classes - these are updated using datasets not used for other classes. 
- Remove observations with classes that have very low representation (less than 5 observations in the class) in the dataset. This is done to allow train_test_split to run. 
- Optionally, join in additional features. feature_list may be modified to select which features to add in (see join_features for further details). 

In [11]:
def data_prep(sample_point_fpath, target):
    """
    Returns dataframe of processed sample points. Sample points are read in from shapefile. Processing includes filtering out null points and developed/agricultural FVTs as 
    well as appending additional features. 
    """

    # Read in gdf
    sample_points = gpd.read_file(sample_point_fpath)
    print(f"Total Number of points: {sample_points.shape[0]}")

    # Filter out near points
    sample_points = sample_points.loc[sample_points['NEAR_FID'] == -1]
    print(f"After removing near points: {sample_points.shape[0]}")

    # Drop unneeeded columns if present
    sample_points = sample_points.drop(['Classified', 'GrndTruth', 'NEAR_FID', 'NEAR_DIST'], axis=1,
                                       errors='ignore')
    
    # Remove observations with -9999/-1111 in any field 
    sample_points = sample_points.loc[~sample_points.isin([-1111, -9999]).any(axis=1)]
    print(f"After removing null points: {sample_points.shape[0]}")

    # Filter out points that weren't disturbed
    sample_points = sample_points.loc[sample_points['LF22_FDST'] != 0]
    print(f"After removing undisturbed points: {sample_points.shape[0]}")

    # Join in additional features
    sample_points = join_features(sample_points)

    # Filter out agricultural and developed points
    developed_fvt = list(range(20,33)) + list(range(2901,2906))
    ag_fvt = [80, 81, 82] + list(range(2960, 2971))
    fvt_filter = developed_fvt + ag_fvt
    sample_points = sample_points.loc[~sample_points['LF22_FVT'].isin(fvt_filter)]

    # Filter out classes in the target feature with very low counts
    sample_points = low_count_filter(sample_points, target)

    return sample_points


## __Instantiate Estimators__

### __Histogram Based Gradient Boosting Classifier__
Scikit-learn implementation of Histogram Based Gradient Boosting Classifier

In [12]:
def histGradientBoostingClassifier(categorical_feature_list, class_weight='Balanced', seed=1234):
    """
    Returns specified histogram-based gradient boosting classifier. 
    """

    hgb_classifier = HistGradientBoostingClassifier(
        categorical_features=categorical_feature_list,  # Natively handle categorical variables
        class_weight=class_weight,
        random_state=seed,
        learning_rate=0.01,
        max_iter=100
    )

    return hgb_classifier

## __Train Model__
Trains and returns HGBC model provided data. Subsets the training data to a provided list of predictors and predicts the specified target feature.

__Note__: Aspect, Elevation, and Slope are the only continuous LANDFIRE datasets, therefore any feature not in that list is assumed to be categorical.

In [16]:
def train_model(train_data, predictors, target, model_type='HGBC'):
    """
    Returns trained model of specified type given a set of predictors. 
    """

    class_weight = "balanced"

    # Get list of predictors for run
    cat_features = [x for x in predictors if x not in ['ASPECT', 'ELEVATION', 'SLOPE']]  # Categorical features for HGBC models  
    
    # Separate training data predictors/response
    y_train = train_data[target].copy()
    X_train = train_data[predictors].copy()

    # Instantiate the model
    model = histGradientBoostingClassifier(categorical_feature_list=cat_features, class_weight=class_weight)
      
    # Fit the model with the training data
    print('Fitting Model...')
    model.fit(X_train, y_train)

    # Return the fit model
    return model         
    

## __Evaluate Model__
Provided a trained model, test data, a set of predictors, and a target - return a list of predictions metrics on prediction accuracy.

In [17]:
def eval_model(model, test_data, predictors, target):
    """
    Returns a dictionary containing 1. 'metrics' : a list of metrics quantifying model performance, and 2. 'predictions' : a list of predictions corresponding to 
    each observation in the sample_points data.
    """
       
    # Separate the predictors from target
    y_test = test_data[target].copy()
    x_test = test_data[predictors].copy()

    # Perform prediction
    y_pred = model.predict(x_test)

    # Get metrics
    accuracy = round(accuracy_score(y_test, y_pred), 3)
    bal_acc = round(balanced_accuracy_score(y_test, y_pred),3)
    recall = round(recall_score(y_test, y_pred, average='macro'), 3)
    precision = round(precision_score(y_test, y_pred, average='macro'), 3)
    f1 = round(f1_score(y_test, y_pred, average='macro'), 3)

    print(f"Predictors: {predictors}")
    print(f"Accuracy: {accuracy}")
    print(f"Balanced Accuracy: {bal_acc}")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F1: {f1}")

    return {
        "metrics" : [accuracy, bal_acc, recall, precision, f1, predictors],
        "predictions" : y_pred
    }


# __Main__

----

## Train Model

In [20]:
# Load in data and perform train/test split.
seed = 1234

# Define predictors and target
predictors = ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']
target = 'LF22_FVT'

# Define path to sample points
sample_points_fpath =  os.path.join(paths_dict['sample_points_dir'], paths_dict['sample_points_fname'])

# Read in and prepare data
sample_points = data_prep(sample_points_fpath, target)

# Define the type of model to be used
model_type = 'HGBC'  

# Train model
model = train_model(train_data=sample_points, predictors=predictors, target=target, model_type=model_type)

# Save model 
datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_fname = f"{target}_{model_type}_model_{datetime}"
model_fpath = os.path.join(paths_dict['model_dir'], model_fname)
dump(model, model_fpath)
print(f"Model written to: {model_fpath}")

Total Number of points: 200390
After removing near points: 196097
After removing null points: 163092
After removing undisturbed points: 163092
Low count LF22_FVT: [2016, 2064, 2066, 2099, 2128, 2177, 2184, 2211, 2219, 2260, 2567, 2910, 2911]
After removing low count LF22_FVT: (160346, 31)
Sample Points Attributes: Index(['LF22_FVT', 'LF20_FVT', 'LF16_FVT', 'LF16_EVT', 'LF14_EVT', 'LF22_FDST',
       'ZONE', 'BPS', 'ASPECT', 'SLOPE', 'ELEVATION', 'LF20_EVT', 'LF20_SCla',
       'LF20_FVH', 'LF20_FVC', 'LF22_FVH', 'LF22_FVC', 'LF16_FVH', 'LF16_FVC',
       'PRED_FVT', 'LF22_F40', 'BPS_FRG_NE', 'geometry', 'BPS_NAME', 'BPS_FRG',
       'FDST_TYPE', 'FDST_SEV', 'FDST_TSD', 'EVT_PHYS', 'EVT_GP_N',
       'EVT_CLASS'],
      dtype='object')
Fitting Model...
Model written to: C:\Users\mcco573\OneDrive - PNNL\Documents\_Projects\BPA Wildfire\Fuel Attributes Model\models\LF22_FVT_HGBC_model_2024-05-16_11-30-32


## Model Quick Evaluation

In [68]:
# Load in data and perform train/test split.
seed = 1234
target = 'LF22_FVT'

## Define train/test split proportion
train_frac = 0.7
test_frac = 0.3

## Define path to sample points
sample_points_fpath =  os.path.join(paths_dict['sample_points_dir'], paths_dict['sample_points_fname'])

## Read in and prepare data
sample_points = data_prep(sample_points_fpath, target)

## Perform Train/Test Split
train, test = train_test_split(sample_points, train_size=train_frac, test_size=test_frac, 
                               random_state=seed, shuffle=True, stratify=sample_points[target])

Total Number of points: 200390
After removing near points: 196097
After removing null points: 163092
After removing undisturbed points: 163092
Low count LF22_FVT: [2016, 2064, 2066, 2099, 2128, 2177, 2184, 2211, 2219, 2260, 2567, 2910, 2911]
After removing low count LF22_FVT: (163057, 31)
Sample Points Attributes: Index(['LF22_FVT', 'LF20_FVT', 'LF16_FVT', 'LF16_EVT', 'LF14_EVT', 'LF22_FDST',
       'ZONE', 'BPS', 'ASPECT', 'SLOPE', 'ELEVATION', 'LF20_EVT', 'LF20_SCla',
       'LF20_FVH', 'LF20_FVC', 'LF22_FVH', 'LF22_FVC', 'LF16_FVH', 'LF16_FVC',
       'PRED_FVT', 'LF22_F40', 'BPS_FRG_NE', 'geometry', 'BPS_NAME', 'BPS_FRG',
       'FDST_TYPE', 'FDST_SEV', 'FDST_TSD', 'EVT_PHYS', 'EVT_GP_N',
       'EVT_CLASS'],
      dtype='object')


In [69]:
# Define the type of model to be used
model_type = 'HGBC'  # RF / HGBC / catBoost / MLP

# Define set of predictors
predictors = ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']

# Train model
model = train_model(train_data=train, predictors=predictors, target=target, model_type=model_type)

# Evaluate Model
print("Score on the Training Set: ")
results = eval_model(model=model, test_data=train, predictors=predictors, target=target, model_type=model_type)

print("Score on the test set: ")
results = eval_model(model=model, test_data=test, predictors=predictors, target=target, model_type=model_type)

Fitting Model...
Score on the Training Set: 
Predictors: ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']
Accuracy: 0.733
Balanced Accuracy: 0.899
Recall: 0.899
Precision: 0.68
F1: 0.756
Score on the test set: 
Predictors: ['LF20_FVT', 'LF22_FDST', 'ZONE', 'ASPECT', 'SLOPE', 'ELEVATION', 'BPS_NAME', 'LF20_FVC', 'LF20_FVH']
Accuracy: 0.67
Balanced Accuracy: 0.558
Recall: 0.558
Precision: 0.462
F1: 0.491


c:\Users\mcco573\AppData\Local\miniconda3\envs\geospatial_310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save out results
## Concatenate predictions to test set
test[f'PRED_{target}'] = results['predictions']

## Create dataframe of metrics
metrics_df = pd.DataFrame(columns=['accuracy', 'balanced accuracy', 'recall', 'precision' ,'f1 score', 'attributes', 'model_type'])
metrics_df.loc[0] = results['metrics'] + [model_type]

## Save out the dataframes
out_dir = make_dir(paths_dict['out_base_dir'], new_dir_name=f'{target}_model_results')
preds_out_fname = f"Predictions_{target}_{model_type}.csv"
test.to_csv(os.path.join(out_dir, preds_out_fname))

metrics_out_fname = f"Metrics_{target}_{model_type}.csv"
metrics_df.to_csv(os.path.join(out_dir, metrics_out_fname))

# Save out model to results dir
datetime = dt.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_fname = f"{target}_{model_type}_model_{datetime}"
dump(model, os.path.join(out_dir, model_fname))